# Download Sentinel-2 data for ASO rasters

Given an ASO raster, find Sentinel-2 scenes over the same area within a week of lidar acqusition, then choose scene with most snow pixels (find max coverage area and least cloud pixels).

In [1]:
# based on exmaples from
# https://planetarycomputer.microsoft.com/docs/tutorials/cloudless-mosaic-sentinel2/
# https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a#Example-Notebook
from pystac.extensions.eo import EOExtension as eo
import pystac_client
import planetary_computer
import glob
import rioxarray as rxr
import re
import datetime
import pandas as pd
from shapely.geometry import box
import odc.stac

In [2]:
def sentinel2_for_aso(aso_raster_fn):
    
    time = pd.to_datetime(re.search("(\d{4}\d{2}\d{2})", aso_raster_fn).group())
    week_before = (time - datetime.timedelta(weeks=1)).strftime('%Y-%m-%d')
    week_after = (time + datetime.timedelta(weeks=1)).strftime('%Y-%m-%d')
    time_of_interest = f'{week_before}/{week_after}'
    
    aso_raster = rxr.open_rasterio(aso_raster_fn).squeeze()
    bounds_latlon = box(*aso_raster.rio.transform_bounds("EPSG:4326"))
    
    
    catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace)

    search = catalog.search(
        collections=["sentinel-2-l2a"],
        intersects=bounds_latlon,
        datetime=time_of_interest)

    # Check how many items were returned
    items = search.item_collection()
    print(f"Returned {len(items)} Items")
    
    sentinel2_stac = odc.stac.load(items,chunks={"x": 2048, "y": 2048},resolution=50)
    sentinel2_stac_clipped = sentinel2_stac.rio.clip_box(*bounds_latlon.bounds,crs="EPSG:4326")
    
    scl = sentinel2_stac_clipped['SCL'].rio.reproject_match(aso_raster).where(aso_raster>=0)
    
    classes = [ #SCL classes here: https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/scene-classification/
    #0,   #No Data (Missing data)	#000000	
    #1,   #Saturated or defective pixel	#ff0000	
    #2,   #Topographic casted shadows (called "Dark features/Shadows" for data before 2022-01-25)	#2f2f2f	
    #3,   #Cloud shadows	#643200	
    4,   #Vegetation	#00a000	
    5,   #Not-vegetated	#ffe65a	
    #6,   #Water	#0000ff	
    #7,   #Unclassified	#808080	
    #8,   #Cloud medium probability	#c0c0c0	
    #9,   #Cloud high probability	#ffffff	
    #10,   #Thin cirrus	#64c8ff	
    11    #Snow or ice      
    ]
    
    idx_least_clouds = scl.where(scl.isin(classes)).sum(dim=['x','y']).idxmax()
    sentinel2_best_lowcloud = sentinel2_stac_clipped.sel(time=idx_least_clouds)
    
    sentinel2_best_lowcloud.to_netcdf(f'/tmp/S2/S2_{pd.to_datetime(idx_least_clouds.values).strftime("%Y%m%d")}_for_{aso_raster_fn.split("/")[-1][:-4]}.nc')

In [10]:
aso_raster_fns = glob.glob('/home/jovyan/crunchy-snow/data/ASO/ASO_50m_SD_cleaned/*/ASO*.tif')

In [7]:
aso_raster_fn

'/home/jovyan/crunchy-snow/data/ASO/ASO_50m_SD_cleaned'

In [13]:
for i, aso_raster_fn in enumerate(aso_raster_fns):
    error_list = []
    print(f'----\nworking on {aso_raster_fn.split("/")[-1]}, {i+1}/{len(aso_raster_fns)}\n----')
    try: 
        sentinel2_for_aso(aso_raster_fn)
    except:
        print('error, skipping')
        error_list.append(aso_raster_fn)

----
working on ASO_50M_SD_BlueRiver_20230529_clean.tif, 1/252
----
Returned 12 Items
----
working on ASO_50M_SD_Animas_20210419_clean.tif, 2/252
----
Returned 24 Items
----
working on ASO_50M_SD_EastRiver_20230523_clean.tif, 3/252
----
Returned 9 Items
----
working on ASO_50M_SD_EastRiver_20220518_clean.tif, 4/252
----
Returned 24 Items
----
working on ASO_50M_SD_SouthPlatte_20230416_clean.tif, 5/252
----
Returned 7 Items
----
working on ASO_50M_SD_USCOCM_20190407_clean.tif, 6/252
----
Returned 3 Items
----
working on ASO_50M_SD_TaylorAndLottis_20230523_clean.tif, 7/252
----
Returned 6 Items
----
working on ASO_50M_SD_USCOGE_20180331_clean.tif, 8/252
----
Returned 24 Items
----
working on ASO_50M_SD_BlueRiver_20190624_clean.tif, 9/252
----
Returned 12 Items
----
working on ASO_50M_SD_BlueRiver_20230416_clean.tif, 10/252
----
Returned 8 Items
----
working on ASO_50M_SD_WindyGap_20230416_clean.tif, 11/252
----
Returned 4 Items
----
working on ASO_50M_SD_RoaringFork_20230528_clean.tif, 1

/srv/conda/envs/notebook/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/srv/conda/envs/notebook/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


----
working on ASO_50M_SD_Truckee_20230511_clean.tif, 224/252
----
Returned 30 Items
----
working on ASO_50M_SD_Truckee_20220310_clean.tif, 225/252
----
Returned 30 Items
----
working on ASO_50M_SD_Feather_20220429_clean.tif, 226/252
----
Returned 33 Items
----
working on ASO_50M_SD_SacramentoMcCloud_20230424_clean.tif, 227/252
----
Returned 24 Items
----
working on ASO_50M_SD_USWAOL_20160208_clean.tif, 228/252
----
Returned 3 Items
----
working on ASO_50M_SD_Yuba_20230531_clean.tif, 229/252
----
Returned 33 Items
----
working on ASO_50M_SD_Truckee_20230409_clean.tif, 230/252
----
Returned 25 Items
----
working on ASO_50M_SD_Feather_20230428_clean.tif, 231/252
----
Returned 29 Items
----
working on ASO_50M_SD_American_20230413_clean.tif, 232/252
----
Returned 30 Items
----
working on ASO_50M_SD_American_20230428_clean.tif, 233/252
----
Returned 30 Items
----
working on ASO_50M_SD_Feather_20230618_clean.tif, 234/252
----
Returned 35 Items
----
working on ASO_50M_SD_Feather_20220310_cle

Aborting load due to failure while reading: https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/10/T/FK/2023/03/30/S2A_MSIL2A_20230330T184951_N0509_R113_T10TFK_20230331T022508.SAFE/GRANULE/L2A_T10TFK_A040579_20230330T185245/IMG_DATA/R20m/T10TFK_20230330T184951_SCL_20m.tif?st=2023-09-27T23%3A06%3A25Z&se=2023-10-05T23%3A06%3A25Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2023-09-28T23%3A06%3A24Z&ske=2023-10-05T23%3A06%3A24Z&sks=b&skv=2021-06-08&sig=rrME7O22L7gOkQzZtwzPh5Q6badP1FS/I0o9Pe3f4M8%3D:1
Aborting load due to failure while reading: https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/10/T/FK/2023/03/30/S2A_MSIL2A_20230330T184951_N0509_R113_T10TFK_20230331T022508.SAFE/GRANULE/L2A_T10TFK_A040579_20230330T185245/IMG_DATA/R20m/T10TFK_20230330T184951_SCL_20m.tif?st=2023-09-27T23%3A06%3A25Z&se=2023-10-05T23%3A06%3A25Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1

error, skipping
----
working on ASO_50M_SD_Yuba_20230625_clean.tif, 240/252
----


Aborting load due to failure while reading: https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/10/T/FK/2023/03/30/S2A_MSIL2A_20230330T184951_N0509_R113_T10TFK_20230331T022508.SAFE/GRANULE/L2A_T10TFK_A040579_20230330T185245/IMG_DATA/R20m/T10TFK_20230330T184951_SCL_20m.tif?st=2023-09-27T23%3A06%3A25Z&se=2023-10-05T23%3A06%3A25Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2023-09-28T23%3A06%3A24Z&ske=2023-10-05T23%3A06%3A24Z&sks=b&skv=2021-06-08&sig=rrME7O22L7gOkQzZtwzPh5Q6badP1FS/I0o9Pe3f4M8%3D:1
Aborting load due to failure while reading: https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/10/T/FK/2023/03/30/S2A_MSIL2A_20230330T184951_N0509_R113_T10TFK_20230331T022508.SAFE/GRANULE/L2A_T10TFK_A040579_20230330T185245/IMG_DATA/R20m/T10TFK_20230330T184951_SCL_20m.tif?st=2023-09-27T23%3A06%3A25Z&se=2023-10-05T23%3A06%3A25Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1

Returned 33 Items
----
working on ASO_50M_SD_USWAOL_20160329_clean.tif, 241/252
----
Returned 4 Items
----
working on ASO_50M_SD_American_20230602_clean.tif, 242/252
----
Returned 36 Items
----
working on ASO_50M_SD_Yuba_20230128_clean.tif, 243/252
----
Returned 28 Items
----
working on ASO_50M_SD_GreenRiver_20220611_clean.tif, 244/252
----
Returned 15 Items
----
working on ASO_50M_SD_Dolores_20230406_clean.tif, 245/252
----
Returned 24 Items
----
working on ASO_50M_SD_USUTLC_20210318_clean.tif, 246/252
----
Returned 12 Items
----
working on ASO_50M_SD_Dolores_20220415_clean.tif, 247/252
----
Returned 24 Items
----
working on ASO_50M_SD_Dolores_20210514_clean.tif, 248/252
----
Returned 24 Items
----
working on ASO_50M_SD_WindRiver_20220611_clean.tif, 249/252
----
Returned 15 Items
----
working on ASO_50M_SD_Dolores_20230525_clean.tif, 250/252
----
Returned 24 Items
----
working on ASO_50M_SD_Dolores_20220510_clean.tif, 251/252
----
Returned 20 Items
----
working on ASO_50M_SD_Dolores_2

In [14]:
aso_raster_fn = '/home/jovyan/crunchy-snow/data/ASO/ASO_50m_SD_cleaned/utm10n/ASO_50M_SD_Yuba_20230405_clean.tif'
sentinel2_for_aso(aso_raster_fn)

Returned 33 Items


Aborting load due to failure while reading: https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/10/T/FK/2023/03/30/S2A_MSIL2A_20230330T184951_N0509_R113_T10TFK_20230331T022508.SAFE/GRANULE/L2A_T10TFK_A040579_20230330T185245/IMG_DATA/R20m/T10TFK_20230330T184951_SCL_20m.tif?st=2023-09-27T23%3A06%3A25Z&se=2023-10-05T23%3A06%3A25Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2023-09-28T23%3A06%3A24Z&ske=2023-10-05T23%3A06%3A24Z&sks=b&skv=2021-06-08&sig=rrME7O22L7gOkQzZtwzPh5Q6badP1FS/I0o9Pe3f4M8%3D:1
Aborting load due to failure while reading: https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/10/T/FK/2023/03/30/S2A_MSIL2A_20230330T184951_N0509_R113_T10TFK_20230331T022508.SAFE/GRANULE/L2A_T10TFK_A040579_20230330T185245/IMG_DATA/R20m/T10TFK_20230330T184951_SCL_20m.tif?st=2023-09-27T23%3A06%3A25Z&se=2023-10-05T23%3A06%3A25Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1

RasterioIOError: '/vsicurl/https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/10/T/FK/2023/03/30/S2A_MSIL2A_20230330T184951_N0509_R113_T10TFK_20230331T022508.SAFE/GRANULE/L2A_T10TFK_A040579_20230330T185245/IMG_DATA/R20m/T10TFK_20230330T184951_SCL_20m.tif?st=2023-09-27T23%3A06%3A25Z&se=2023-10-05T23%3A06%3A25Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2023-09-28T23%3A06%3A24Z&ske=2023-10-05T23%3A06%3A24Z&sks=b&skv=2021-06-08&sig=rrME7O22L7gOkQzZtwzPh5Q6badP1FS/I0o9Pe3f4M8%3D' does not exist in the file system, and is not recognized as a supported dataset name.